# Web Search Agents with Strands

**Building Intelligent Web-Enabled AI Agents**

---

Welcome to this comprehensive tutorial on creating **web search agents** with the Strands framework! This notebook demonstrates how to build powerful AI agents that can search the web, gather real-time information, and synthesize knowledge from multiple sources. By the end of this 10-minute tutorial, you'll have built an intelligent research assistant. **You will need a Google account**.

### 🎯 What You'll Learn

In this tutorial, you will:
- Set up Google Custom Search API integration
- Build a custom web search tool
- Create intelligent agents that know when to search
- Implement source citation and synthesis
- Handle real-time information queries
- Optimize performance and error handling

### 🔍 Why Web Search Agents?

Web search agents extend AI capabilities by:
- Accessing **current information** beyond training data
- Providing **factual, up-to-date responses**
- Enabling **research and comparison** tasks
- Supporting **real-time data** queries
- Offering **verifiable sources** for information

## 📦 Step 1: Installing Required Packages

### Overview
We'll need the Strands SDK and Google's API client library for this tutorial.

### 📚 Packages We'll Install
- **strands-agents**: Core agent framework
- **google-api-python-client**: Google Custom Search API client
- **strands-agents-tools**: Additional tool utilities

In [ ]:
# Install required packages
%pip install strands-agents strands-agents-tools strands-agents-builder google-api-python-client -q

print("✅ All packages installed successfully!")
print("   Ready to build web-enabled agents! 🌐")

## 🔑 Step 2: Setting Up Google Search API

### Overview
To enable web search, we need to set up Google Custom Search API. This involves getting an API key and creating a search engine.

### 📝 Quick Setup Guide

#### 1. Get Your API Key
1. Visit [Google Custom Search API](https://developers.google.com/custom-search/v1/introduction)
2. Click **"Get a Key"** button
3. Create a project (e.g., "StrandsSearchAgent")
4. Copy your API key
5. Store it in a file called "google-api-key.txt" in the directory where this notebook is stored.

#### 2. Create Search Engine
1. Go to [Programmable Search Engine](https://programmablesearchengine.google.com/controlpanel/create)
2. Select **"Search the entire web"**
3. Complete the setup
4. Copy your Search Engine ID
5. Store it in a file called "google-search_engine-id.txt" in the directory where this notebook is stored.

### 🔐 Security Note
Store your credentials in separate files, never in code! In my case, I use the following files (as shown in the code below):
 - "google-api-key.txt" for the Google API key.
 - "google-search_engine-id.txt" for the Search Engine ID.

In [ ]:
# Configure file paths for credentials
GOOGLE_API_KEY_FILE = "./google-api-key.txt"
SEARCH_ENGINE_ID_FILE = "./google-search_engine-id.txt"

# Load credentials
try:
    with open(GOOGLE_API_KEY_FILE, 'r') as f:
        google_api_key = f.read().strip()
    print("✅ Google API key loaded successfully!")
except FileNotFoundError:
    print(f"❌ Please create {GOOGLE_API_KEY_FILE} with your API key")
    google_api_key = None

try:
    with open(SEARCH_ENGINE_ID_FILE, 'r') as f:
        search_engine_id = f.read().strip()
    print("✅ Search Engine ID loaded successfully!")
except FileNotFoundError:
    print(f"❌ Please create {SEARCH_ENGINE_ID_FILE} with your Search Engine ID")
    search_engine_id = None

if google_api_key and search_engine_id:
    print("\n🎉 All credentials loaded! Ready to search the web.")
else:
    print("\n⚠️  Missing credentials. Please complete setup before continuing.")

## 🔐 Step 3: Setting Up AWS Authentication

### Overview
We'll configure AWS Bedrock to power our AI agents with Claude 3.7 Sonnet.

### 🔑 Authentication Options
1. **AWS Bedrock API Keys** (EASIEST - Recommended for beginners)
2. **AWS Profile** (Traditional method)
3. **Environment Variables**
4. **IAM Roles** (Recommended for production)

In [ ]:
# 🚀 EASIEST OPTION: Paste your Bedrock API Key here
# Get your API key from: https://console.aws.amazon.com/bedrock/ -> API keys
API_KEY = ""  # Paste your API key between the quotes

# Configuration constants (you can modify these if needed)
REGION_NAME = "us-west-2"
AWS_PROFILE = "default"

# Set up authentication using our utility function
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / "src"))  # Add the src directory to path for imports
from auth_utils import setup_bedrock_auth, display_auth_status

auth_status = setup_bedrock_auth(
    api_key=API_KEY,
    region_name=REGION_NAME,
    aws_profile=AWS_PROFILE
)

# Display the authentication status
display_auth_status(auth_status)

# Import required modules
from strands import Agent, tool
from googleapiclient.discovery import build
import time

print("🔧 Authentication setup complete!")
print("   Ready to create web search agents!")

## 🔧 Step 4: Creating the Google Search Tool

### Tool Overview
This custom tool integrates Google Custom Search with Strands, enabling agents to search the web intelligently.

### 🚀 Key Features
- **Configurable Results**: 1-10 search results per query
- **Rich Formatting**: Title, URL, and description for each result
- **Error Handling**: Graceful handling of API failures
- **Type Safety**: Proper type hints for reliable execution

In [ ]:
@tool
def google_search(query: str, num_results: int = 3) -> str:
    """
    Search Google and return top results with titles, URLs, and snippets.
    
    Args:
        query: The search query string
        num_results: Number of results to return (default: 3, max: 10)
    
    Returns:
        Formatted string with search results including titles, URLs, and descriptions
    """
    # Validate credentials
    if not google_api_key or not search_engine_id:
        return "❌ Error: Google API credentials not configured."
    
    # Validate and limit results
    num_results = max(1, min(num_results, 10))
    
    try:
        # Build the Custom Search service
        service = build("customsearch", "v1", developerKey=google_api_key)
        
        # Perform the search
        result = service.cse().list(
            q=query,
            cx=search_engine_id,
            num=num_results
        ).execute()
        
        # Check if results were found
        if 'items' not in result:
            return f"🔍 No search results found for query: '{query}'"
        
        # Format the results
        formatted_results = []
        for i, item in enumerate(result['items'][:num_results], 1):
            title = item.get('title', 'No title')
            link = item.get('link', 'No URL')
            snippet = item.get('snippet', 'No description')
            
            formatted_result = f"**Result {i}:**\n"
            formatted_result += f"📋 Title: {title}\n"
            formatted_result += f"🔗 URL: {link}\n"
            formatted_result += f"📝 Description: {snippet}\n"
            
            formatted_results.append(formatted_result)
        
        return "\n".join(formatted_results)
        
    except Exception as e:
        return f"❌ Error performing search: {str(e)}"

print("🔧 Google search tool created successfully!")
print("   Features: Configurable results, rich formatting, error handling")

## 🧪 Step 5: Testing the Search Tool

### Direct Testing
Before integrating with an agent, let's test our search tool directly to ensure it's working correctly.

In [ ]:
# Test the search tool directly
test_query = "latest AI developments 2024"
print(f"🔍 Testing search with query: '{test_query}'")
print("=" * 60)

search_results = google_search(test_query, num_results=2)
print(search_results)

print("\n" + "=" * 60)
print("✅ Search tool test completed!")

## 🤖 Step 6: Creating the Search Agent

### Agent Design
Our search agent will intelligently decide when to search the web and how to synthesize information from multiple sources.

### 🧠 Agent Capabilities
- **Smart Search Decisions**: Knows when web search is needed
- **Source Citation**: Always provides URLs and references
- **Information Synthesis**: Combines multiple sources coherently
- **Context Awareness**: Tailors responses to user needs

In [ ]:
# Create an intelligent search agent
search_agent = Agent(
    system_prompt="""You are a helpful research assistant with Google search capabilities.
    
    🔍 WHEN TO SEARCH:
    - Current information or recent news
    - Facts that may have changed
    - Real-time data (weather, stocks, events)
    - Comparative information
    - Specific products, services, or locations
    
    📋 RESPONSE GUIDELINES:
    - Always cite sources with URLs
    - Synthesize information from multiple sources
    - Be concise but comprehensive
    - Indicate when information is time-sensitive
    - Acknowledge limitations if results are unclear
    
    ✅ QUALITY STANDARDS:
    - Prioritize authoritative sources
    - Present information objectively
    - Distinguish facts from opinions
    - Provide context when necessary""",
    tools=[google_search],
    model=auth_status.strands_bedrock_model
)

print("🤖 Search agent created successfully!")
print("   Capabilities: Web search, source citation, information synthesis")
print(f"   Authentication: {auth_status['method']}")
print(f"   Region: {auth_status['region']}")

## 🎯 Step 7: Testing Different Query Types

Let's test our search agent with various types of queries that require web search.

In [ ]:
# Test various query types
test_questions = [
    "What are the latest developments in AI in 2024?",
    "What is the current weather in Tokyo?",
]

print("🎯 Testing search agent with various queries...\n")

for i, question in enumerate(test_questions, 1):
    print(f"{'='*80}")
    print(f"📌 Query {i}: {question}")
    print("\n")
    
    # Measure response time
    start_time = time.time()
    response = search_agent(question)
    response_time = time.time() - start_time
    print(f"\n{'='*80}\n")

## 📊 Step 8: Performance Analysis

### Understanding Performance
Let's analyze the performance characteristics of our search tool to understand optimization opportunities.

In [ ]:
def analyze_search_performance(query: str):
    """Analyze performance of search operations"""
    print(f"📊 Performance Analysis for: '{query}'")
    print("-" * 60)
    
    # Test direct tool performance
    print("🔧 Direct tool call...")
    start_time = time.time()
    direct_result = google_search(query, num_results=3)
    direct_time = time.time() - start_time
    
    # Test agent performance
    print("🤖 Agent with tool...")
    start_time = time.time()
    agent_result = search_agent(f"Search for: {query}")
    agent_time = time.time() - start_time
    
    # Calculate metrics
    ai_overhead = agent_time - direct_time
    
    print(f"\n📊 Results:")
    print(f"   Direct tool: {direct_time:.2f}s")
    print(f"   Agent call: {agent_time:.2f}s")
    print(f"   AI overhead: {ai_overhead:.2f}s")
    print(f"   Overhead %: {(ai_overhead/agent_time*100):.1f}%")
    
    return direct_time, agent_time

# Run analysis
direct_time, agent_time = analyze_search_performance("Python best practices 2024")

## ⚠️ Step 9: Error Handling and Edge Cases

### Robustness Testing
Let's ensure our search agent handles edge cases and errors gracefully.

In [ ]:
# Test edge cases
edge_cases = [
    ("", "Empty query"),
    ("asdfghjklqwertyuiopzxcvbnm", "Nonsensical query"),
    ("site:nonexistentwebsite12345.com test", "No results expected")
]

print("⚠️ Testing Edge Cases")

for query, description in edge_cases:
    print("=" * 60)
    print(f"\n📌 Test: {description}")
    print(f"   Query: '{query}'")
    
    response = search_agent(f"Search for: {query}")
    print("\n" + ("=" * 60) + "\n")

print("\n✅ Edge case testing completed!")

## 🎓 Advanced: Multi-Step Research Assistant

### Combining Multiple Searches
Let's create a sophisticated research workflow that performs multiple searches and synthesizes comprehensive reports.

In [ ]:
# Create a specialized research assistant
research_assistant = Agent(
    system_prompt="""You are an expert research analyst specializing in technology trends.
    
    🔍 RESEARCH METHODOLOGY:
    - Perform multiple targeted searches for comprehensive coverage
    - Cross-reference information from different sources
    - Identify patterns and trends
    - Provide balanced, well-researched analysis
    
    📊 REPORT STRUCTURE:
    - Executive Summary
    - Key Findings (with sources)
    - Detailed Analysis
    - Recommendations
    - Source Citations
    
    ✅ QUALITY STANDARDS:
    - Use recent, authoritative sources
    - Provide specific examples and data
    - Balance different perspectives
    - Clearly cite all sources""",
    tools=[google_search],
    model=auth_status.strands_bedrock_model
)

# Complex research task
research_task = """Research the impact of AI on software development in 2024. 
Include:
1. Current AI coding tools and their adoption rates
2. Impact on developer productivity
3. Future trends and predictions
"""

print("🎓 ADVANCED RESEARCH TASK")
print("=" * 100)
print(f"📋 Task: {research_task}")
print("=" * 100)

# Perform research
start_time = time.time()
report = research_assistant(research_task)
total_time = time.time() - start_time

print("\n📊 Research Report:")
print(report)
print(f"\n⏱️  Total Research Time: {total_time:.2f} seconds")

## 🎉 Congratulations!

### 🏆 What You've Accomplished
In this tutorial, you've learned how to:
- ✅ Set up Google Custom Search API
- ✅ Build a custom web search tool
- ✅ Create intelligent search agents
- ✅ Handle real-time information queries
- ✅ Synthesize information from multiple sources
- ✅ Implement error handling and edge cases
- ✅ Optimize performance

### 🚀 Key Features Built

1. **Smart Search Tool**: Configurable, formatted, error-handled
2. **Intelligent Agent**: Knows when and how to search
3. **Source Citation**: Always provides verifiable references
4. **Information Synthesis**: Combines multiple sources coherently
5. **Performance Optimization**: Efficient API usage

### 💡 Best Practices

1. **API Key Security**: Always store credentials separately
2. **Rate Limiting**: Be mindful of API quotas
3. **Error Handling**: Gracefully handle API failures
4. **Source Quality**: Prioritize authoritative sources
5. **Context Awareness**: Tailor searches to user needs

### 🔮 Enhancement Ideas

Consider adding:
- **Multiple Search Engines**: Bing, DuckDuckGo integration
- **Specialized Searches**: News, images, videos
- **Result Caching**: Avoid duplicate searches
- **Advanced Filtering**: Date ranges, domains
- **Fact Checking**: Cross-reference multiple sources

### 📚 Resources

- [Strands Documentation](https://strandsagents.com/0.1.x/)
- [AWS Bedrock API Keys Guide](https://docs.aws.amazon.com/bedrock/latest/userguide/api-keys.html)
- [Google Custom Search API](https://developers.google.com/custom-search/)
- [AWS Bedrock Guide](https://docs.aws.amazon.com/bedrock/)

### 🌟 Next Steps

You're now ready to:
1. Build specialized research agents for any domain
2. Create real-time information systems
3. Develop fact-checking applications
4. Build competitive intelligence tools
5. Create news aggregation systems

The web is now at your agent's fingertips!

Happy searching! 🔍🤖✨